In [3]:
import numpy as np
from json import load
import spacy
import nltk
import string
from nltk.corpus import stopwords
from string import punctuation
from spacy.matcher import PhraseMatcher
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, recall_score, f1_score, precision_score
import joblib
nlp = spacy.load("ru_core_news_sm")


In [4]:
not_discriptors = list()
with open('../../Data/Words/word_pairs_not_disc.txt', 'r', encoding='utf-8') as txt:
    for i in txt:
        not_discriptors.append(i.replace('\n', ''))

with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    events = load(js)

with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discriptors = load(js)


In [5]:
dict_of_disc = {'expession': [], 'is_discr': []}
for subject in discriptors.values():
    for d in sum(subject.values(), []):
        dict_of_disc['expession'].append(nlp(d).vector)
        dict_of_disc['is_discr'].append(True)

for subject in not_discriptors:
    dict_of_disc['expession'].append(nlp(subject).vector)
    dict_of_disc['is_discr'].append(False)



In [6]:
features = np.array(dict_of_disc['expession'])
labels = np.array(list(map(int, dict_of_disc['is_discr'])))


In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=71)


In [21]:
X_train.shape

(798, 96)

# Подбираем гиппер-параметры.

In [11]:
logreg_optimal_f1 = GridSearchCV(LogisticRegression(solver='saga'),
                                                        {
                                                            'penalty': ['l1', 'l2', None, 'elasticnet'],
                                                            'C': np.arange(0.1, 2, 0.1)
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)
logreg_optimal_precission = GridSearchCV(LogisticRegression(solver='saga'),
                                                        {
                                                            'penalty': ['l1', 'l2', None, 'elasticnet'],
                                                            'C': np.arange(0.1, 2, 0.1)
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)
logreg_optimal_recall = GridSearchCV(LogisticRegression(solver='saga'),
                                                        {
                                                            'penalty': ['l1', 'l2', None, 'elasticnet'],
                                                            'C': np.arange(0.1, 2, 0.1)
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 positional arguments but 3 were given

  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\mo

## Формула Байеса

In [17]:
y_predicted_by_logreg_f1 = logreg_optimal_f1.predict(X_train)
y_predicted_by_logreg_precission = logreg_optimal_precission.predict(X_test)
y_predicted_by_logreg_recall = logreg_optimal_recall.predict(X_test)

## Оценка качества 

In [19]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_logreg_f1))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_logreg_f1))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_logreg_f1))



Accuracy
0.9138576779026217
Confusion matrix
[[ 82  14]
 [  9 162]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.90      0.85      0.88        96
           1       0.92      0.95      0.93       171

    accuracy                           0.91       267
   macro avg       0.91      0.90      0.91       267
weighted avg       0.91      0.91      0.91       267



In [14]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_logreg_precission))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_logreg_precission))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_logreg_precission))



Accuracy
0.9138576779026217
Confusion matrix
[[ 82  14]
 [  9 162]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.90      0.85      0.88        96
           1       0.92      0.95      0.93       171

    accuracy                           0.91       267
   macro avg       0.91      0.90      0.91       267
weighted avg       0.91      0.91      0.91       267



In [15]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_logreg_recall))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_logreg_recall))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_logreg_recall))



Accuracy
0.9138576779026217
Confusion matrix
[[ 82  14]
 [  9 162]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.90      0.85      0.88        96
           1       0.92      0.95      0.93       171

    accuracy                           0.91       267
   macro avg       0.91      0.90      0.91       267
weighted avg       0.91      0.91      0.91       267



## All

In [25]:
r = np.append(X_train, X_test, axis=0)
rr = np.append(y_test, y_train)

In [26]:
y_predicted_by_logreg_f1 = logreg_optimal_f1.predict(r)
y_predicted_by_logreg_precission = logreg_optimal_precission.predict(r)
y_predicted_by_logreg_recall = logreg_optimal_recall.predict(r)

In [27]:
print('Accuracy')
print(accuracy_score(rr, y_predicted_by_logreg_f1))
print('Confusion matrix')
print(confusion_matrix(rr, y_predicted_by_logreg_f1))
print('Precision, Recall, F\n', classification_report(
    rr, y_predicted_by_logreg_f1))



Accuracy
0.5615023474178403
Confusion matrix
[[108 250]
 [217 490]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.33      0.30      0.32       358
           1       0.66      0.69      0.68       707

    accuracy                           0.56      1065
   macro avg       0.50      0.50      0.50      1065
weighted avg       0.55      0.56      0.56      1065



In [28]:
print('Accuracy')
print(accuracy_score(rr, y_predicted_by_logreg_precission))
print('Confusion matrix')
print(confusion_matrix(rr, y_predicted_by_logreg_precission))
print('Precision, Recall, F\n', classification_report(
    rr, y_predicted_by_logreg_precission))



Accuracy
0.5615023474178403
Confusion matrix
[[108 250]
 [217 490]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.33      0.30      0.32       358
           1       0.66      0.69      0.68       707

    accuracy                           0.56      1065
   macro avg       0.50      0.50      0.50      1065
weighted avg       0.55      0.56      0.56      1065



In [29]:
print('Accuracy')
print(accuracy_score(rr, y_predicted_by_logreg_recall))
print('Confusion matrix')
print(confusion_matrix(rr, y_predicted_by_logreg_recall))
print('Precision, Recall, F\n', classification_report(
    rr, y_predicted_by_logreg_recall))



Accuracy
0.5615023474178403
Confusion matrix
[[108 250]
 [217 490]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.33      0.30      0.32       358
           1       0.66      0.69      0.68       707

    accuracy                           0.56      1065
   macro avg       0.50      0.50      0.50      1065
weighted avg       0.55      0.56      0.56      1065

